In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.formula.api as smf
from scipy.stats import bernoulli
from scipy.stats import truncnorm

In [5]:
import math

In [31]:
def DGP(intervene_A=None):
    p = 0.6     # Above, this was defined as 0.5. In paper, it's 0.6
    n = 10000

    # A (gender)
    if intervene_A:
        A2 = np.ones(n)
    else:
        A2 = U_A = bernoulli.rvs(p=p, size=n)


    # Q (qualifications)
    U_Q = np.random.normal(2, 5, n)
    Q2 = np.floor(U_Q)


    # D (number of children)
    lower_d, upper_d = 0.1, 3
    mu_d, sigma_d = 2, 1
    X_d = stats.truncnorm((lower_d - mu_d) / sigma_d, (upper_d - mu_d) / sigma_d, loc=mu_d, scale=sigma_d)
    U_D = X_d.rvs(n)
    D2 = A2 + np.floor(0.5 * Q2 * U_D)


    # M (physical strength)
    lower_m, upper_m = 0.1, 3
    mu_m, sigma_m = 3, 2
    X_m = stats.truncnorm((lower_m - mu_m) / sigma_m, (upper_m - mu_m) / sigma_m, loc=mu_m, scale=sigma_m)
    U_M = X_m.rvs(n)
    M2 = 3*A2 + (0.4 * Q2 * U_M)


    sigmoid = lambda x: 1/(1+ math.exp(-x))

    Y2 = np.zeros(n)
    for i in range(n):
        Y2[i] = sigmoid(-10+5*A2[i]+Q2[i]+D2[i]+M2[i]) >= 0.5


    df = pd.DataFrame({'A': A2, 'D': D2, 'M': M2 ,'Q': Q2,'Y':Y2})
    return df

In [32]:
without_intervention = DGP(intervene_A= False)
intervention = DGP(intervene_A= True)

In [ ]:
#1. why did qualification change?
#2. for the counterfactual prediction, should I just use the q from without intervention dataset?
#3. Evaluating on fairness metric on test dataset?

In [34]:
without_intervention.head()

,A,D,M,Q,Y,bi_Q
0,1,1.0,3.256999,1.0,1.0,1
1,1,1.0,3.369268,1.0,1.0,1
2,1,5.0,6.542736,5.0,1.0,-1
3,0,10.0,6.208922,8.0,1.0,-1
4,1,0.0,1.926975,-1.0,0.0,1


In [35]:
intervention.head()

,A,D,M,Q,Y,bi_Q
0,1.0,1.0,5.277394,2.0,1.0,1
1,1.0,11.0,7.458967,9.0,1.0,-1
2,1.0,3.0,7.988663,5.0,1.0,-1
3,1.0,-7.0,-3.123909,-7.0,0.0,1
4,1.0,2.0,3.590503,1.0,1.0,1


In [40]:
Counterfactual_df.head()  #generate myself by holding value of Q unchanged at individual level, and switched A 1->0, 0->1

,A,D,M,Q,Y
0,0,0.0,0.211016,1.0,0.0
1,0,0.0,1.051374,1.0,0.0
2,0,2.0,3.452420,5.0,1.0
3,1,12.0,6.083610,8.0,1.0
4,0,-2.0,-1.022882,-1.0,0.0


# Generate Counterfactual Data

In [33]:
without_intervention['bi_Q'] = np.where(without_intervention['Q'] <= np.quantile(without_intervention['Q'], 0.5), 1, -1)
intervention['bi_Q'] = np.where(intervention['Q'] <= np.quantile(intervention['Q'], 0.5), 1, -1)
#intervention['Q'] = without_intervention['Q']
#intervention['bi_Q'] = without_intervention['bi_Q']


In [39]:
#counterfactual gender
n = 10000
A2 = 1 - without_intervention.A.values

# Q (qualifications) - unchanged
    #U_Q = np.random.normal(2, 5, n)
    #Q2 = np.floor(U_Q)
Q2 = without_intervention.Q.values


# D (number of children)
lower_d, upper_d = 0.1, 3
mu_d, sigma_d = 2, 1
X_d = stats.truncnorm((lower_d - mu_d) / sigma_d, (upper_d - mu_d) / sigma_d, loc=mu_d, scale=sigma_d)
U_D = X_d.rvs(n)
D2 = A2 + np.floor(0.5 * Q2 * U_D)


# M (physical strength)
lower_m, upper_m = 0.1, 3
mu_m, sigma_m = 3, 2
X_m = stats.truncnorm((lower_m - mu_m) / sigma_m, (upper_m - mu_m) / sigma_m, loc=mu_m, scale=sigma_m)
U_M = X_m.rvs(n)
M2 = 3*A2 + (0.4 * Q2 * U_M)


sigmoid = lambda x: 1/(1+ math.exp(-x))

Y2 = np.zeros(n)
for i in range(n):
    Y2[i] = sigmoid(-10+5*A2[i]+Q2[i]+D2[i]+M2[i]) >= 0.5


Counterfactual_df = pd.DataFrame({'A': A2, 'D': D2, 'M': M2 ,'Q': Q2,'Y':Y2})

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#Full model

full_list = ['A', 'Q', 'D', 'M']
X_train, X_test, y_train, y_test = train_test_split(without_intervention[full_list], without_intervention.Y,
                                                   test_size = 0.2, 
                                                   stratify = without_intervention.Y)

In [65]:
Count_X_train = Counterfactual_df[full_list].iloc[X_train.index.values, :]
Count_X_test = Counterfactual_df[full_list].iloc[X_test.index.values, :]
Count_y_train = Counterfactual_df['Y'][X_train.index.values]
Counter_y_test = Counterfactual_df['Y'][X_test.index.values]

# Logistic Regression

## Full

In [66]:
lr_full = LogisticRegression()
lr_full.fit(X_train, y_train)

X_train['X_train_pred'] = lr_full.predict(X_train)
Count_X_train['counter_f_trn_pred'] = lr_full.predict(Count_X_train)
X_test['X_test_pred'] = lr_full.predict(X_test)
Count_X_test['counter_f_tst_pred'] = lr_full.predict(Count_X_test)

In [89]:
lr_full.score(X_test[full_list], y_test)

0.997

In [81]:
pd.DataFrame({ 'female (A = 0)': [
    len(X_test[X_test['A'] == 0][X_test['X_test_pred'] == 1])/len(X_test[X_test['A'] == 0]),
    len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_f_tst_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
],
              'male (A = 1)':[
    len(X_test[X_test['A'] == 1][X_test['X_test_pred'] == 1])/len(X_test[X_test['A'] == 1]),
    len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_f_tst_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])
              ]              
}, index = ['Factual', 'Counterfactual']
).T

<ipython-input-81-8c619c860c82>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 0][X_test['X_test_pred'] == 1])/len(X_test[X_test['A'] == 0]),
<ipython-input-81-8c619c860c82>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_f_tst_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
<ipython-input-81-8c619c860c82>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 1][X_test['X_test_pred'] == 1])/len(X_test[X_test['A'] == 1]),
<ipython-input-81-8c619c860c82>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_f_tst_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])


,Factual,Counterfactual
female (A = 0),0.322621,0.587145
male (A = 1),0.574307,0.296390


## Unaware

In [84]:
#Unaware
unaware = ['Q', 'D', 'M']
lr_unaware = LogisticRegression()
lr_unaware.fit(X_train[unaware], y_train)

0.973375

In [88]:
lr_unaware.score(X_test[unaware], y_test)

0.9745

In [86]:
X_train['unaware_pred'] = lr_unaware.predict(X_train[unaware])
Count_X_train['counter_unaware_trn_pred'] = lr_unaware.predict(Count_X_train[unaware])
X_test['X_test_unaware_pred'] = lr_unaware.predict(X_test[unaware])
Count_X_test['counter_unaware_tst_pred'] = lr_unaware.predict(Count_X_test[unaware])

In [87]:
pd.DataFrame({ 'female (A = 0)': [
    len(X_test[X_test['A'] == 0][X_test['X_test_unaware_pred'] == 1])/len(X_test[X_test['A'] == 0]),
    len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_unaware_tst_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
],
              'male (A = 1)':[
    len(X_test[X_test['A'] == 1][X_test['X_test_unaware_pred'] == 1])/len(X_test[X_test['A'] == 1]),
    len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_unaware_tst_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])
              ]              
}, index = ['Factual', 'Counterfactual']
).T


<ipython-input-87-384d6ca421c2>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 0][X_test['X_test_unaware_pred'] == 1])/len(X_test[X_test['A'] == 0]),
<ipython-input-87-384d6ca421c2>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_unaware_tst_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
<ipython-input-87-384d6ca421c2>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 1][X_test['X_test_unaware_pred'] == 1])/len(X_test[X_test['A'] == 1]),
<ipython-input-87-384d6ca421c2>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_unaware_tst_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])


,Factual,Counterfactual
female (A = 0),0.336218,0.576020
male (A = 1),0.557515,0.319899


## Decision Tree Classifier

In [90]:
from sklearn.tree import DecisionTreeClassifier

dt_full = DecisionTreeClassifier()
dt_unaware = DecisionTreeClassifier()

dt_full.fit(X_train[full_list], y_train)
dt_unaware.fit(X_train[unaware], y_train)

DecisionTreeClassifier()

In [92]:
print(dt_full.score(X_test[full_list], y_test))
print(dt_unaware.score(X_test[unaware], y_test))

0.999
0.994


In [93]:
X_train['dt_full_pred'] = dt_full.predict(X_train[full_list])
Count_X_train['counter_dt_full_pred'] = dt_full.predict(Count_X_train[full_list])

X_test['X_test_dt_full_pred'] = dt_full.predict(X_test[full_list])
Count_X_test['counter_dt_full_pred'] = dt_full.predict(Count_X_test[full_list])

In [94]:
X_train['dt_unaware_pred'] = dt_unaware.predict(X_train[unaware])
Count_X_train['counter_unaware_dt_pred'] = dt_unaware.predict(Count_X_train[unaware])

X_test['X_test_dt_unaware_pred'] = dt_unaware.predict(X_test[unaware])
Count_X_test['counter_unaware_dt_pred'] = dt_unaware.predict(Count_X_test[unaware])

## Full

In [95]:
pd.DataFrame({ 'female (A = 0)': [
    len(X_test[X_test['A'] == 0][X_test['X_test_dt_full_pred'] == 1])/len(X_test[X_test['A'] == 0]),
    len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_dt_full_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
],
              'male (A = 1)':[
    len(X_test[X_test['A'] == 1][X_test['X_test_dt_full_pred'] == 1])/len(X_test[X_test['A'] == 1]),
    len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_dt_full_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])
              ]              
}, index = ['Factual', 'Counterfactual']
).T

<ipython-input-95-f5ab2e8aa46b>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 0][X_test['X_test_dt_full_pred'] == 1])/len(X_test[X_test['A'] == 0]),
<ipython-input-95-f5ab2e8aa46b>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_dt_full_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
<ipython-input-95-f5ab2e8aa46b>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 1][X_test['X_test_dt_full_pred'] == 1])/len(X_test[X_test['A'] == 1]),
<ipython-input-95-f5ab2e8aa46b>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_dt_full_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])


,Factual,Counterfactual
female (A = 0),0.316440,0.588381
male (A = 1),0.575147,0.293871


## Unaware

In [96]:
pd.DataFrame({ 'female (A = 0)': [
    len(X_test[X_test['A'] == 0][X_test['X_test_dt_unaware_pred'] == 1])/len(X_test[X_test['A'] == 0]),
    len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_unaware_dt_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
],
              'male (A = 1)':[
    len(X_test[X_test['A'] == 1][X_test['X_test_dt_unaware_pred'] == 1])/len(X_test[X_test['A'] == 1]),
    len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_unaware_dt_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])
              ]              
}, index = ['Factual', 'Counterfactual']
).T

<ipython-input-96-455c109ff28f>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 0][X_test['X_test_dt_unaware_pred'] == 1])/len(X_test[X_test['A'] == 0]),
<ipython-input-96-455c109ff28f>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_unaware_dt_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
<ipython-input-96-455c109ff28f>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 1][X_test['X_test_dt_unaware_pred'] == 1])/len(X_test[X_test['A'] == 1]),
<ipython-input-96-455c109ff28f>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_unaware_dt_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])


,Factual,Counterfactual
female (A = 0),0.320148,0.585909
male (A = 1),0.570949,0.298908


## SVM Classifier

In [97]:
from sklearn import svm

svm_full = svm.SVC()
svm_unaware = svm.SVC()

svm_full.fit(X_train[full_list], y_train)
svm_unaware.fit(X_train[unaware], y_train)

SVC()

In [98]:
print(svm_full.score(X_test[full_list], y_test))
print(svm_unaware.score(X_test[unaware], y_test))

0.99
0.9825


In [99]:
X_train['svm_full_pred'] = svm_full.predict(X_train[full_list])
Count_X_train['counter_full_svm_pred'] = svm_full.predict(Count_X_train[full_list])

X_test['X_test_svm_full_pred'] = svm_full.predict(X_test[full_list])
Count_X_test['counter_full_svm_pred'] = svm_full.predict(Count_X_test[full_list])

In [100]:
X_train['svm_unaware_pred'] = svm_unaware.predict(X_train[unaware])
Count_X_train['counter_unaware_svm_pred'] = svm_unaware.predict(Count_X_train[unaware])

X_test['X_test_svm_unaware_pred'] = svm_unaware.predict(X_test[unaware])
Count_X_test['counter_unaware_svm_pred'] = svm_unaware.predict(Count_X_test[unaware])

## Full

In [101]:
pd.DataFrame({ 'female (A = 0)': [
    len(X_test[X_test['A'] == 0][X_test['X_test_svm_full_pred'] == 1])/len(X_test[X_test['A'] == 0]),
    len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_full_svm_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
],
              'male (A = 1)':[
    len(X_test[X_test['A'] == 1][X_test['X_test_svm_full_pred'] == 1])/len(X_test[X_test['A'] == 1]),
    len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_full_svm_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])
              ]              
}, index = ['Factual', 'Counterfactual']
).T

<ipython-input-101-57ada0f83553>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 0][X_test['X_test_svm_full_pred'] == 1])/len(X_test[X_test['A'] == 0]),
<ipython-input-101-57ada0f83553>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_full_svm_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
<ipython-input-101-57ada0f83553>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 1][X_test['X_test_svm_full_pred'] == 1])/len(X_test[X_test['A'] == 1]),
<ipython-input-101-57ada0f83553>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_full_svm_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])


,Factual,Counterfactual
female (A = 0),0.331273,0.585909
male (A = 1),0.570109,0.304786


## Unaware

In [102]:
pd.DataFrame({ 'female (A = 0)': [
    len(X_test[X_test['A'] == 0][X_test['X_test_svm_unaware_pred'] == 1])/len(X_test[X_test['A'] == 0]),
    len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_unaware_svm_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
],
              'male (A = 1)':[
    len(X_test[X_test['A'] == 1][X_test['X_test_svm_unaware_pred'] == 1])/len(X_test[X_test['A'] == 1]),
    len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_unaware_svm_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])
              ]              
}, index = ['Factual', 'Counterfactual']
).T

<ipython-input-102-e2d29d6902d2>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 0][X_test['X_test_svm_unaware_pred'] == 1])/len(X_test[X_test['A'] == 0]),
<ipython-input-102-e2d29d6902d2>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 1][Count_X_test['counter_unaware_svm_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 1])
<ipython-input-102-e2d29d6902d2>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(X_test[X_test['A'] == 1][X_test['X_test_svm_unaware_pred'] == 1])/len(X_test[X_test['A'] == 1]),
<ipython-input-102-e2d29d6902d2>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(Count_X_test[Count_X_test['A'] == 0][Count_X_test['counter_unaware_svm_pred'] == 1])/len(Count_X_test[Count_X_test['A'] == 0])


,Factual,Counterfactual
female (A = 0),0.338690,0.580964
male (A = 1),0.564232,0.319899
